# AI Guardrails and Safety Controls

This notebook teaches you how to add **guardrails** and **control mechanisms** to AI agents and models, ensuring safe, compliant, and reliable AI behavior in production systems.

## What are Guardrails?

Guardrails are safety mechanisms that:
- **Prevent harmful outputs** - Block dangerous, offensive, or inappropriate content
- **Enforce business rules** - Ensure AI adheres to organizational policies
- **Validate inputs/outputs** - Check data quality and format
- **Control behavior** - Limit AI actions and decision-making scope
- **Ensure compliance** - Meet regulatory and legal requirements

## Why Guardrails are Critical

Without guardrails, AI systems can:
- ❌ Generate harmful or biased content
- ❌ Leak sensitive information
- ❌ Make unauthorized decisions
- ❌ Violate compliance requirements
- ❌ Damage brand reputation

## Types of Controls

```
┌─────────────────────────────────────────────────────────┐
│                    Control Layers                        │
│                                                          │
│  ┌──────────────┐      ┌──────────────┐               │
│  │   Input      │─────▶│  Processing  │─────▶         │
│  │ Validation   │      │  Filters     │               │
│  └──────────────┘      └──────────────┘               │
│         │                      │                        │
│         ▼                      ▼                        │
│  ┌──────────────┐      ┌──────────────┐               │
│  │   Content    │◀─────│    Output    │               │
│  │   Safety     │      │  Validation  │               │
│  └──────────────┘      └──────────────┘               │
│         │                      │                        │
│         └──────────┬───────────┘                        │
│                    ▼                                    │
│            ┌──────────────┐                            │
│            │   Logging &  │                            │
│            │  Monitoring  │                            │
│            └──────────────┘                            │
└─────────────────────────────────────────────────────────┘
```

## Control Mechanisms in Semantic Kernel

1. **Filters** - Intercept and modify kernel/function invocations
2. **Prompt Filters** - Control what goes to the AI model
3. **Function Filters** - Control function execution
4. **Auto Function Filters** - Control automatic function calling
5. **Content Safety** - Azure Content Safety integration
6. **Custom Validators** - Business-specific validation logic

## References

- [Semantic Kernel Filters Documentation](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion/filters)
- [Azure Content Safety](https://learn.microsoft.com/azure/ai-services/content-safety/)
- [Responsible AI Practices](https://www.microsoft.com/ai/responsible-ai)

## Setup and Configuration

In [1]:
// Load configuration settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

Console.WriteLine($"Configuration loaded:");
Console.WriteLine($"- Model: {model}");
Console.WriteLine($"- Endpoint: {azureEndpoint}");
Console.WriteLine($"- API Key configured: {!string.IsNullOrEmpty(apiKey)}");

Configuration loaded:
- Model: gpt-4o
- Endpoint: https://eastus.api.cognitive.microsoft.com/
- API Key configured: True


## Install Required Packages

In [2]:
// Install Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.67.1"

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;  // Alias to avoid conflicts
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
using System.Text.RegularExpressions;
using System.Text.Json;

Console.WriteLine("Packages loaded successfully.");

Installed Packages Microsoft.SemanticKernel, 1.67.1

Packages loaded successfully.


## Part 1: Prompt Filters - Input Control

Prompt filters intercept prompts **before** they're sent to the AI model. Use them to:
- Block inappropriate inputs
- Sanitize user input
- Add system instructions
- Log prompts for auditing

### Example 1: Content Filtering

In [3]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;  // Alias to avoid conflicts

// Prompt filter that blocks inappropriate content
public class ContentSafetyPromptFilter : IPromptRenderFilter
{
    private static readonly string[] _bannedWords = new[]
    {
        "password", "secret", "credit card", "ssn", "hack", "exploit"
    };

    public async Task OnPromptRenderAsync(
        PromptRenderContext context, 
        Func<PromptRenderContext, Task> next)
    {
        Console.WriteLine($"\n[Prompt Filter] Checking input safety...");
        
        // Get the rendered prompt
        var renderedPrompt = context.RenderedPrompt?.ToLower() ?? "";
        
        // Check for banned words
        var foundBannedWords = _bannedWords
            .Where(word => renderedPrompt.Contains(word, StringComparison.OrdinalIgnoreCase))
            .ToList();
        
        if (foundBannedWords.Any())
        {
            Console.WriteLine($"[Prompt Filter] ❌ BLOCKED - Found banned words: {string.Join(", ", foundBannedWords)}");
            
            // Block the request
            throw new InvalidOperationException(
                $"Content policy violation: Input contains prohibited terms ({string.Join(", ", foundBannedWords)})");
        }
        
        Console.WriteLine($"[Prompt Filter] ✓ Input passed safety check");
        
        // Call next filter in chain (or proceed to AI model)
        await next(context);
    }
}

Console.WriteLine("Content Safety Prompt Filter defined.");

Content Safety Prompt Filter defined.


### Example 2: PII Detection Filter

In [4]:
// Filter that detects and redacts Personally Identifiable Information (PII)
public class PIIDetectionFilter : IPromptRenderFilter
{
    // Simple regex patterns for common PII
    private static readonly Regex _emailPattern = new Regex(
        @"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",
        RegexOptions.Compiled);
    
    private static readonly Regex _phonePattern = new Regex(
        @"\b\d{3}[-.]?\d{3}[-.]?\d{4}\b",
        RegexOptions.Compiled);
    
    private static readonly Regex _ssnPattern = new Regex(
        @"\b\d{3}-\d{2}-\d{4}\b",
        RegexOptions.Compiled);

    public async Task OnPromptRenderAsync(
        PromptRenderContext context,
        Func<PromptRenderContext, Task> next)
    {
        Console.WriteLine($"\n[PII Filter] Scanning for PII...");
        
        var prompt = context.RenderedPrompt ?? "";
        var originalPrompt = prompt;
        bool piiFound = false;
        
        // Detect and redact emails
        if (_emailPattern.IsMatch(prompt))
        {
            prompt = _emailPattern.Replace(prompt, "[EMAIL_REDACTED]");
            piiFound = true;
            Console.WriteLine($"[PII Filter] ⚠️  Email detected and redacted");
        }
        
        // Detect and redact phone numbers
        if (_phonePattern.IsMatch(prompt))
        {
            prompt = _phonePattern.Replace(prompt, "[PHONE_REDACTED]");
            piiFound = true;
            Console.WriteLine($"[PII Filter] ⚠️  Phone number detected and redacted");
        }
        
        // Detect and redact SSN
        if (_ssnPattern.IsMatch(prompt))
        {
            prompt = _ssnPattern.Replace(prompt, "[SSN_REDACTED]");
            piiFound = true;
            Console.WriteLine($"[PII Filter] ⚠️  SSN detected and redacted");
        }
        
        if (piiFound)
        {
            Console.WriteLine($"[PII Filter] Modified prompt to remove PII");
            // Update the context with redacted prompt
            // Note: In SK 1.67.1, you'd need to modify the ChatHistory directly
        }
        else
        {
            Console.WriteLine($"[PII Filter] ✓ No PII detected");
        }
        
        await next(context);
    }
}

Console.WriteLine("PII Detection Filter defined.");

PII Detection Filter defined.


## Part 2: Function Calling Filters

Function filters control **what functions** can be called and **how** they're executed.

### Example: Function Authorization Filter

In [5]:
// Filter that controls which functions can be executed
public class FunctionAuthorizationFilter : IFunctionInvocationFilter
{
    private static readonly HashSet<string> _allowedFunctions = new()
    {
        "GetWeather",
        "SearchDocuments",
        "GetUserProfile"
    };
    
    private static readonly HashSet<string> _restrictedFunctions = new()
    {
        "DeleteUser",
        "ModifyDatabase",
        "SendEmail",
        "ExecuteCommand"
    };

    public async Task OnFunctionInvocationAsync(
        FunctionInvocationContext context,
        Func<FunctionInvocationContext, Task> next)
    {
        var functionName = context.Function.Name;
        Console.WriteLine($"\n[Function Filter] Function call requested: {functionName}");
        
        // Check if function is explicitly restricted
        if (_restrictedFunctions.Contains(functionName))
        {
            Console.WriteLine($"[Function Filter] ❌ BLOCKED - Function '{functionName}' is restricted");
            
            // Block the function call
            throw new UnauthorizedAccessException(
                $"Function '{functionName}' is not authorized for execution");
        }
        
        // Optionally, only allow explicitly whitelisted functions
        // Uncomment to enable strict whitelisting:
        /*
        if (!_allowedFunctions.Contains(functionName))
        {
            Console.WriteLine($"[Function Filter] ❌ BLOCKED - Function '{functionName}' not in whitelist");
            throw new UnauthorizedAccessException(
                $"Function '{functionName}' is not in the allowed list");
        }
        */
        
        Console.WriteLine($"[Function Filter] ✓ Function '{functionName}' authorized");
        
        // Log function parameters (be careful with sensitive data)
        Console.WriteLine($"[Function Filter] Parameters: {context.Arguments.Count} argument(s)");
        
        // Execute the function
        await next(context);
        
        // Log result (optional)
        Console.WriteLine($"[Function Filter] Function '{functionName}' completed successfully");
    }
}

Console.WriteLine("Function Authorization Filter defined.");

Function Authorization Filter defined.


### Example: Function Rate Limiting Filter

In [6]:
// Filter that implements rate limiting for function calls
public class FunctionRateLimitFilter : IFunctionInvocationFilter
{
    private readonly Dictionary<string, List<DateTime>> _callHistory = new();
    private readonly int _maxCallsPerMinute = 5;
    private readonly TimeSpan _timeWindow = TimeSpan.FromMinutes(1);

    public async Task OnFunctionInvocationAsync(
        FunctionInvocationContext context,
        Func<FunctionInvocationContext, Task> next)
    {
        var functionName = context.Function.Name;
        var now = DateTime.UtcNow;
        
        // Get call history for this function
        if (!_callHistory.ContainsKey(functionName))
        {
            _callHistory[functionName] = new List<DateTime>();
        }
        
        var calls = _callHistory[functionName];
        
        // Remove calls outside the time window
        calls.RemoveAll(time => now - time > _timeWindow);
        
        // Check rate limit
        if (calls.Count >= _maxCallsPerMinute)
        {
            Console.WriteLine($"\n[Rate Limit] ❌ BLOCKED - Function '{functionName}' exceeded rate limit");
            Console.WriteLine($"[Rate Limit] {calls.Count} calls in last {_timeWindow.TotalSeconds} seconds");
            
            throw new InvalidOperationException(
                $"Rate limit exceeded for function '{functionName}'. " +
                $"Max {_maxCallsPerMinute} calls per {_timeWindow.TotalMinutes} minute(s).");
        }
        
        // Record this call
        calls.Add(now);
        
        Console.WriteLine($"\n[Rate Limit] ✓ Function '{functionName}' within rate limit ({calls.Count}/{_maxCallsPerMinute})");
        
        await next(context);
    }
}

Console.WriteLine("Function Rate Limit Filter defined.");

Function Rate Limit Filter defined.


## Part 3: Output Validation Filters

These filters check AI-generated content **after** it's produced but **before** it's returned to the user.

In [23]:
// Filter that validates and sanitizes AI outputs
public class OutputValidationFilter : IAutoFunctionInvocationFilter
{
    private static readonly string[] _toxicWords = new[]
    {
        "offensive_word1", "offensive_word2" // Add actual words as needed
    };

    public async Task OnAutoFunctionInvocationAsync(
        AutoFunctionInvocationContext context,
        Func<AutoFunctionInvocationContext, Task> next)
    {
        // Execute function first
        await next(context);
        
        // Check the result
        var result = context.Result?.ToString() ?? "";
        
        Console.WriteLine($"\n[Output Filter] Validating output...");
        
        // Check for toxic content
        var foundToxicWords = _toxicWords
            .Where(word => result.Contains(word, StringComparison.OrdinalIgnoreCase))
            .ToList();
        
        if (foundToxicWords.Any())
        {
            Console.WriteLine($"[Output Filter] ⚠️  Toxic content detected, sanitizing...");
            
            // Sanitize the output
            foreach (var word in foundToxicWords)
            {
                result = result.Replace(word, "[REMOVED]", StringComparison.OrdinalIgnoreCase);
            }
            
            // You could also completely block the output:
            // throw new InvalidOperationException("Output contains inappropriate content");
        }
        
        // Check output length
        if (result.Length > 5000)
        {
            Console.WriteLine($"[Output Filter] ⚠️  Output too long ({result.Length} chars), truncating...");
            result = result.Substring(0, 5000) + "... [truncated]";
        }
        
        // Validate JSON if expected
        if (context.Function.Metadata.ReturnParameter?.ParameterType?.Name == "JsonDocument")
        {
            try
            {
                JsonDocument.Parse(result);
                Console.WriteLine($"[Output Filter] ✓ Valid JSON output");
            }
            catch
            {
                Console.WriteLine($"[Output Filter] ❌ Invalid JSON output detected");
                throw new InvalidOperationException("Function returned invalid JSON");
            }
        }
        
        Console.WriteLine($"[Output Filter] ✓ Output validated");
    }
}

Console.WriteLine("Output Validation Filter defined.");

Output Validation Filter defined.


## Part 4: Comprehensive Logging and Monitoring Filter

In [24]:
// Filter that logs all interactions for compliance and debugging
public class AuditLoggingFilter : IPromptRenderFilter, IFunctionInvocationFilter
{
    private readonly List<AuditLog> _auditLogs = new();

    public async Task OnPromptRenderAsync(
        PromptRenderContext context,
        Func<PromptRenderContext, Task> next)
    {
        var startTime = DateTime.UtcNow;
        
        Console.WriteLine($"\n[Audit Log] Prompt rendering started at {startTime:HH:mm:ss.fff}");
        
        try
        {
            await next(context);
            
            var duration = DateTime.UtcNow - startTime;
            
            _auditLogs.Add(new AuditLog
            {
                Timestamp = startTime,
                EventType = "PromptRender",
                Success = true,
                Duration = duration,
                Details = new Dictionary<string, object>
                {
                    ["PromptLength"] = context.RenderedPrompt?.Length ?? 0,
                    ["FunctionName"] = context.Function.Name
                }
            });
            
            Console.WriteLine($"[Audit Log] ✓ Prompt rendered in {duration.TotalMilliseconds:F2}ms");
        }
        catch (Exception ex)
        {
            var duration = DateTime.UtcNow - startTime;
            
            _auditLogs.Add(new AuditLog
            {
                Timestamp = startTime,
                EventType = "PromptRender",
                Success = false,
                Duration = duration,
                Error = ex.Message
            });
            
            Console.WriteLine($"[Audit Log] ❌ Prompt rendering failed: {ex.Message}");
            throw;
        }
    }

    public async Task OnFunctionInvocationAsync(
        FunctionInvocationContext context,
        Func<FunctionInvocationContext, Task> next)
    {
        var startTime = DateTime.UtcNow;
        var functionName = context.Function.Name;
        
        Console.WriteLine($"\n[Audit Log] Function '{functionName}' invocation started");
        
        try
        {
            await next(context);
            
            var duration = DateTime.UtcNow - startTime;
            
            _auditLogs.Add(new AuditLog
            {
                Timestamp = startTime,
                EventType = "FunctionInvocation",
                Success = true,
                Duration = duration,
                Details = new Dictionary<string, object>
                {
                    ["FunctionName"] = functionName,
                    ["ArgumentCount"] = context.Arguments.Count
                }
            });
            
            Console.WriteLine($"[Audit Log] ✓ Function executed in {duration.TotalMilliseconds:F2}ms");
        }
        catch (Exception ex)
        {
            var duration = DateTime.UtcNow - startTime;
            
            _auditLogs.Add(new AuditLog
            {
                Timestamp = startTime,
                EventType = "FunctionInvocation",
                Success = false,
                Duration = duration,
                Error = ex.Message,
                Details = new Dictionary<string, object>
                {
                    ["FunctionName"] = functionName
                }
            });
            
            Console.WriteLine($"[Audit Log] ❌ Function failed: {ex.Message}");
            throw;
        }
    }
    
    public void PrintAuditReport()
    {
        Console.WriteLine($"\n" + "=" + new string('=', 60));
        Console.WriteLine("AUDIT REPORT");
        Console.WriteLine("=" + new string('=', 60));
        Console.WriteLine($"Total Events: {_auditLogs.Count}");
        Console.WriteLine($"Successful: {_auditLogs.Count(l => l.Success)}");
        Console.WriteLine($"Failed: {_auditLogs.Count(l => !l.Success)}");
        Console.WriteLine($"Avg Duration: {_auditLogs.Average(l => l.Duration.TotalMilliseconds):F2}ms");
        Console.WriteLine("=" + new string('=', 60));
    }
}

public class AuditLog
{
    public DateTime Timestamp { get; set; }
    public string EventType { get; set; } = "";
    public bool Success { get; set; }
    public TimeSpan Duration { get; set; }
    public string? Error { get; set; }
    public Dictionary<string, object>? Details { get; set; }
}

Console.WriteLine("Audit Logging Filter defined.");

Audit Logging Filter defined.


## Part 5: Putting It All Together - Secure Kernel Configuration

In [25]:
using Microsoft.Extensions.DependencyInjection;

// Create a kernel with all security filters enabled
var secureKernelBuilder = Kernel.CreateBuilder();

// Add AI service
secureKernelBuilder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

// Add all filters in order
var auditFilter = new AuditLoggingFilter();

secureKernelBuilder.Services.AddSingleton<IPromptRenderFilter>(new ContentSafetyPromptFilter());
secureKernelBuilder.Services.AddSingleton<IPromptRenderFilter>(new PIIDetectionFilter());
secureKernelBuilder.Services.AddSingleton<IPromptRenderFilter>(auditFilter);

// Option 2: Add filter directly to the Kernel instance after the Build()
//secureKernel.PromptRenderFilters.Add(new PromprtRenderFilter())

secureKernelBuilder.Services.AddSingleton<IFunctionInvocationFilter>(new FunctionAuthorizationFilter());
secureKernelBuilder.Services.AddSingleton<IFunctionInvocationFilter>(new FunctionRateLimitFilter());
secureKernelBuilder.Services.AddSingleton<IFunctionInvocationFilter>(auditFilter);

// Option 2: Add filter directly to the Kernel instance after the Build()
//kernel.FunctionInvocationFilters.Add(new FunctionInvocationFilter());

secureKernelBuilder.Services.AddSingleton<IAutoFunctionInvocationFilter>(new OutputValidationFilter());
// Option 2: Add filter directly to the Kernel instance after the Build()
//kernel.AutoFunctionInvocationFilters.Add(new AutFunctionInvocationFilter());

// Build the secure kernel
var secureKernel = secureKernelBuilder.Build();

Console.WriteLine("\n" + "=" + new string('=', 60));
Console.WriteLine("SECURE KERNEL CONFIGURED");
Console.WriteLine("=" + new string('=', 60));
Console.WriteLine("Enabled Filters:");
Console.WriteLine("✓ Content Safety Prompt Filter");
Console.WriteLine("✓ PII Detection Filter");
Console.WriteLine("✓ Function Authorization Filter");
Console.WriteLine("✓ Function Rate Limit Filter");
Console.WriteLine("✓ Output Validation Filter");
Console.WriteLine("✓ Audit Logging Filter");
Console.WriteLine("=" + new string('=', 60));


SECURE KERNEL CONFIGURED
Enabled Filters:
✓ Content Safety Prompt Filter
✓ PII Detection Filter
✓ Function Authorization Filter
✓ Function Rate Limit Filter
✓ Output Validation Filter
✓ Audit Logging Filter


## Part 6: Testing the Guardrails

Let's test our security filters with various scenarios:

### Test 1: Safe Request (Should Pass)

In [26]:
Console.WriteLine("\n" + "=" + new string('=', 60));
Console.WriteLine("TEST 1: Safe Request");
Console.WriteLine("=" + new string('=', 60));

try
{
    var result = await secureKernel.InvokePromptAsync(
        "What is the weather like today?");
    
    Console.WriteLine($"\nResult: {result}");
    Console.WriteLine("\n✓ Request completed successfully");
}
catch (Exception ex)
{
    Console.WriteLine($"\n❌ Request blocked: {ex.Message}");
}


TEST 1: Safe Request

[Function Filter] Function call requested: InvokePromptAsync_ae8b1e294e3e4139a55f1346ef8d4547
[Function Filter] ✓ Function 'InvokePromptAsync_ae8b1e294e3e4139a55f1346ef8d4547' authorized
[Function Filter] Parameters: 0 argument(s)

[Rate Limit] ✓ Function 'InvokePromptAsync_ae8b1e294e3e4139a55f1346ef8d4547' within rate limit (1/5)

[Audit Log] Function 'InvokePromptAsync_ae8b1e294e3e4139a55f1346ef8d4547' invocation started

[Prompt Filter] Checking input safety...
[Prompt Filter] ✓ Input passed safety check

[PII Filter] Scanning for PII...
[PII Filter] ✓ No PII detected

[Audit Log] Prompt rendering started at 05:39:50.564
[Audit Log] ✓ Prompt rendered in 0.12ms
[Audit Log] ✓ Function executed in 2577.08ms
[Function Filter] Function 'InvokePromptAsync_ae8b1e294e3e4139a55f1346ef8d4547' completed successfully

Result: I'm unable to provide real-time information, such as today's weather, as my data only goes up to October 2023 and I don't have live internet access.

### Test 2: Request with Banned Words (Should be Blocked)

In [27]:
Console.WriteLine("\n" + "=" + new string('=', 60));
Console.WriteLine("TEST 2: Request with Banned Words");
Console.WriteLine("=" + new string('=', 60));

try
{
    var result = await secureKernel.InvokePromptAsync(
        "How do I hack into a system?");
    
    Console.WriteLine($"\nResult: {result}");
    Console.WriteLine("\n⚠️  Request should have been blocked!");
}
catch (Exception ex)
{
    Console.WriteLine($"\n✓ Request blocked as expected: {ex.Message}");
}


TEST 2: Request with Banned Words

[Function Filter] Function call requested: InvokePromptAsync_d204055363424768b62f2fd6f1a3d7b6
[Function Filter] ✓ Function 'InvokePromptAsync_d204055363424768b62f2fd6f1a3d7b6' authorized
[Function Filter] Parameters: 0 argument(s)

[Rate Limit] ✓ Function 'InvokePromptAsync_d204055363424768b62f2fd6f1a3d7b6' within rate limit (1/5)

[Audit Log] Function 'InvokePromptAsync_d204055363424768b62f2fd6f1a3d7b6' invocation started

[Prompt Filter] Checking input safety...
[Prompt Filter] ✓ Input passed safety check

[PII Filter] Scanning for PII...
[PII Filter] ✓ No PII detected

[Audit Log] Prompt rendering started at 05:39:53.259
[Audit Log] ✓ Prompt rendered in 0.04ms
[Audit Log] ✓ Function executed in 1147.77ms
[Function Filter] Function 'InvokePromptAsync_d204055363424768b62f2fd6f1a3d7b6' completed successfully

Result: I'm sorry, but I can't assist with that.

⚠️  Request should have been blocked!


### Test 3: Request with PII (Should be Redacted)

In [28]:
Console.WriteLine("\n" + "=" + new string('=', 60));
Console.WriteLine("TEST 3: Request with PII");
Console.WriteLine("=" + new string('=', 60));

try
{
    var result = await secureKernel.InvokePromptAsync(
        "Send an email to john.doe@example.com about the meeting.");
    
    Console.WriteLine($"\nResult: {result}");
    Console.WriteLine("\n✓ PII was detected and handled");
}
catch (Exception ex)
{
    Console.WriteLine($"\n❌ Request blocked: {ex.Message}");
}


TEST 3: Request with PII

[Function Filter] Function call requested: InvokePromptAsync_1064eae8f4844b698c191bb0cf4241a4
[Function Filter] ✓ Function 'InvokePromptAsync_1064eae8f4844b698c191bb0cf4241a4' authorized
[Function Filter] Parameters: 0 argument(s)

[Rate Limit] ✓ Function 'InvokePromptAsync_1064eae8f4844b698c191bb0cf4241a4' within rate limit (1/5)

[Audit Log] Function 'InvokePromptAsync_1064eae8f4844b698c191bb0cf4241a4' invocation started

[Prompt Filter] Checking input safety...
[Prompt Filter] ✓ Input passed safety check

[PII Filter] Scanning for PII...
[PII Filter] ✓ No PII detected

[Audit Log] Prompt rendering started at 05:39:54.549
[Audit Log] ✓ Prompt rendered in 0.10ms
[Audit Log] ❌ Function failed: HTTP 400 (: content_filter)
Parameter: prompt

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https:/

### View Audit Report

In [29]:
// Print comprehensive audit report
auditFilter.PrintAuditReport();


AUDIT REPORT
Total Events: 6
Successful: 5
Failed: 1
Avg Duration: 1309.78ms


## Part 7: Advanced Patterns

### Pattern 1: Context-Aware Filtering

In [30]:
// Filter that adjusts behavior based on user context
public class ContextAwareFilter : IPromptRenderFilter
{
    public enum UserRole { Admin, PowerUser, BasicUser }
    
    private UserRole _currentUserRole = UserRole.BasicUser;

    public async Task OnPromptRenderAsync(
        PromptRenderContext context,
        Func<PromptRenderContext, Task> next)
    {
        Console.WriteLine($"\n[Context Filter] Checking permissions for {_currentUserRole}");
        
        var prompt = context.RenderedPrompt ?? "";
        
        // Basic users have stricter filtering
        if (_currentUserRole == UserRole.BasicUser)
        {
            // Add safety instructions
            var safetyPrefix = "[System: Respond carefully. Do not provide dangerous or harmful information.]\n";
            // In production, you'd modify the context here
            
            Console.WriteLine($"[Context Filter] Added safety instructions for BasicUser");
        }
        else if (_currentUserRole == UserRole.Admin)
        {
            Console.WriteLine($"[Context Filter] Admin mode - full access granted");
        }
        
        await next(context);
    }
}

Console.WriteLine("Context-Aware Filter defined.");

Context-Aware Filter defined.


### Pattern 2: Token Budget Control

In [31]:
// Filter that enforces token budgets to control costs
public class TokenBudgetFilter : IPromptRenderFilter
{
    private int _tokensUsedToday = 0;
    private readonly int _dailyTokenBudget = 100000;
    private DateTime _lastReset = DateTime.UtcNow.Date;

    public async Task OnPromptRenderAsync(
        PromptRenderContext context,
        Func<PromptRenderContext, Task> next)
    {
        // Reset counter daily
        if (DateTime.UtcNow.Date > _lastReset)
        {
            _tokensUsedToday = 0;
            _lastReset = DateTime.UtcNow.Date;
            Console.WriteLine($"\n[Token Budget] Daily token counter reset");
        }
        
        // Estimate tokens (rough approximation: 1 token ≈ 4 characters)
        var prompt = context.RenderedPrompt ?? "";
        var estimatedTokens = prompt.Length / 4;
        
        Console.WriteLine($"\n[Token Budget] Estimated tokens: {estimatedTokens}");
        Console.WriteLine($"[Token Budget] Used today: {_tokensUsedToday}/{_dailyTokenBudget}");
        
        if (_tokensUsedToday + estimatedTokens > _dailyTokenBudget)
        {
            Console.WriteLine($"[Token Budget] ❌ Daily budget exceeded!");
            throw new InvalidOperationException(
                $"Daily token budget of {_dailyTokenBudget} exceeded. " +
                $"Used: {_tokensUsedToday}, Requested: {estimatedTokens}");
        }
        
        await next(context);
        
        // Track usage (in production, get actual token count from response)
        _tokensUsedToday += estimatedTokens;
        Console.WriteLine($"[Token Budget] Updated usage: {_tokensUsedToday}/{_dailyTokenBudget}");
    }
}

Console.WriteLine("Token Budget Filter defined.");

Token Budget Filter defined.


## Key Takeaways

### 1. **Layered Security**
- Multiple filters provide defense in depth
- Each filter has a specific responsibility
- Filters can be combined and ordered

### 2. **Filter Types**
- **Prompt Filters** - Control inputs before AI
- **Function Filters** - Control function execution
- **Auto Function Filters** - Control automatic function calling
- **Output Filters** - Validate AI responses

### 3. **Common Use Cases**
- ✅ Content safety and moderation
- ✅ PII detection and redaction
- ✅ Function authorization
- ✅ Rate limiting
- ✅ Audit logging
- ✅ Token budget control
- ✅ Context-aware filtering

### 4. **Best Practices**
- Always log security events
- Fail securely (block when in doubt)
- Test filters with adversarial inputs
- Monitor filter performance
- Keep filter logic simple and focused
- Update filters as threats evolve

### 5. **Production Considerations**
- Use Azure Content Safety for enterprise-grade filtering
- Store audit logs in persistent storage
- Implement alerting for blocked requests
- Regular security reviews
- User education about limitations

## Next Steps

- Explore [Azure Content Safety](https://learn.microsoft.com/azure/ai-services/content-safety/)
- Implement custom filters for your use case
- Set up monitoring and alerting
- Create incident response procedures
- Regular security audits

## Resources

- [SK Filters Documentation](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion/filters)
- [Responsible AI Guidelines](https://www.microsoft.com/ai/responsible-ai)
- [Azure Content Safety](https://azure.microsoft.com/products/ai-services/ai-content-safety)